In [22]:
#Import packaeds that is needed in the project
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from IPython.display import Image

In [44]:
#import the ratings csv files and get information of the dataset
data = pd.read_csv('ratings.csv')
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 610.4 MB


In [45]:
#import movies dataset and get information 
mov = pd.read_csv('movies.csv')
mov.head()
mov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [32]:
#reshape the ratings dataset turn the dataset into a matrix as userID be the row, and movieID be the column and rating be the value
rat = data.pivot(index='userId',columns='movieId',values='rating')

ValueError: ignored

In [25]:
#process the dataset and change NaNs values to 0 so all the values are ready to use
rat= data.fillna(0).values

In [33]:
#define the train dataset and split the training dataset for prediction and validation to check the data


'''
Parameters of nonZeros: Means this only takes if the user has rated over that particular value. 
              SizeRate: the randomised size of ratings for validation 
'''
def train_test_split(ratings, nonZeros, sizeRate):
    #validation is to check whether the input data is correct or not and if there is any invalid balues
    validation = np.zeros(rat.shape)
    train = ratings.copy() 
    
    for user in np.arange(rat.shape[0]):
        if len(rat[user,:].nonzero()[0])>=nonZeros:
            val_ratings = np.random.choice(rat[user, :].nonzero()[0], 
                                        size=sizeRate, 
                                        replace=False)
            train[user, val_ratings] = 0
            validation[user, val_ratings] = rat[user, val_ratings]
    return train, validation


In [35]:
##Defining the distance functions of RMSE and MAE to calculate the error of the prediction 
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

def mae(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return mean_absolute_error(prediction, ground_truth)
#apply gradient discent to the lower rank matrices and update the matrices to the correspond best fit
#W is the item feature matrix and V is the user matrix
def prediction(V,W):
    return np.dot(V.T,W)

In [42]:
#create a procress bar for the massive loops in order to help us to look at the progress and better manage it

from tqdm import tnrange, tqdm_notebook
#define the final prediction with the result from above 
def prediction_final():
    #uses the selected best parameters that has been used before
    train, val = train_test_split(rat, 35, 15)
    #regularization parameters that makes sure the matrix has a full rank and can be solveable for later use and has a unique solution
    lmbda = 1e-3 
    k = 2 
    # size of rhe train data where m is the users and n is the items
    m, n = train.shape 
    #the number of epochs that is going to run, how many iterations it is going to apply 
    n_epochs = 100  
    #learning rate for the model and it will affect the speed of the convergence and the performance of the model
    alpha=0.01 
    #user feature matrix for latent,initial value
    V = 3 * np.random.rand(k,m) 
    # movie feature matrix for latent,initial value
    W = 3 * np.random.rand(k,n)

    
    #in the prediction of train and test only take rating into consideration
    users,items = train.nonzero()      
    for epoch in tqdm_notebook(range(n_epochs)):
        for u, i in zip(users,items):
            #e is the error that is goinf to be used in the gradient update
            e = train[u, i] - prediction(V[:,u],W[:,i])  
            #update the user feature matrix based on e which is calculated above and regularization parameter
            V[:,u] += alpha * ( e * W[:,i] - lmbda * V[:,u]) 
            #updated the item features based on e and regularization parameter which makes sure to have a unique solution
            W[:,i] += alpha * ( e * V[:,u] - lmbda * W[:,i])  
            # print out result for each iteration 
        print(f"Epoch number: {epoch}, training rmse: {rmse(prediction(V,W),train)}, Testing RMSE: {rmse(prediction(V,W),val)}, MAE: {mae(prediction(V,W),val)}")

    return(prediction(V,W))



In [43]:
#print the result 
pred = predction_final()


NameError: ignored

In [40]:
# define get movies name for output recommendation system
def get_movie_name(index):
    return(mov['title'].values[index])

In [41]:
##GETTING THE TOP 5 MOVIES FOR A PARTICULAR USER:
def getMovies(userID , ratings = rat, prediction = pred, movies = mov):
    user = rat[userID]
    movie_ids = np.nonzero(user)
    list_of_movies = []
    list_of_alrady_liked = []
    ratingAbove4 = []
    for i in movie_ids[0]:
        list_of_movies.append(get_movie_name(i))
        list_of_alrady_liked.append(i)
        #only select movie ratings that is aboe 4
        if user[i] >4.5:
          #get movie names
            ratingAbove4.append(get_movie_name(i))
    
    list_user_pred = pred[userID]
    movie_dict = {}
    for i in range(len(list_user_pred)):
        if list_user_pred[i] > 4.5:
            movie_dict[list_user_pred[i]] = (i, get_movie_name(i))
    #output the top 5 movies
    
    top_movies = []
    for j in sorted(movie_dict.keys())[::-1]:
        if movie_dict[j][0] not in list_of_alrady_liked:
            top_movies.append(movie_dict[j])
    
    
    if len(ratingAbove4) >=11:
        return(ratingAbove4[:10], top_movies[:5]) 
    else:
        return(ratingAbove4, top_movies[:5])
        
    

NameError: ignored

In [ ]:
#print out movies recommendation
getMovies(11)